In [1]:
import os
#os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

os.environ['TF_USE_LEGACY_KERAS'] = '1';

In [2]:
input_default_json_conf_file='fcnn_emotion4_kfold_default.json';

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
import json

2024-08-08 18:28:33.914630: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-08 18:28:35.215017: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import sys
sys.path.append('../library');

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-08-08 18:28:36.954415: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-08 18:28:37.300809: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-08 18:28:37.301600: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
## Load json conf json file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

# Variable globales

In [7]:
## Seed for the random variables
seed_number = 0;

## Dataset 
dataset_base_dir    = DATA["dataset_base_dir"]; 
dataset_labels_file = DATA['dataset_labels_file'];
dataset_name        = DATA['dataset_name'];

## Kfold 
K                 = DATA["kfold"]; # Variable K of kfold
enable_stratified = DATA["enable_stratified"]; # True: Stratified kfold False: Enable kfold 

## Training hyperparameters
EPOCAS     = DATA["epochs"];
BATCH_SIZE = DATA["batch_size"];

## Output
output_base_dir = DATA["output_base_dir"];

## Output json file
fold_status_file='kfold_data_results.json';

patience=max(10,int(EPOCAS/5));
NCOD=15;
ANGLE=60;

# Parametros de entrada

In [8]:
for n in range(len(sys.argv)):
    if sys.argv[n]=='--dataset-dir':
        dataset_base_dir=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-file':
        dataset_labels_file=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-name':
        dataset_name=sys.argv[n+1];
    elif sys.argv[n]=='--kfold':
        K=int(sys.argv[n+1]);
    elif sys.argv[n]=='--enable-stratified':
        enable_stratified=bool(sys.argv[n+1]);
    elif sys.argv[n]=='--epochs':
        EPOCAS=int(sys.argv[n+1]);
    elif sys.argv[n]=='--batch-size':
        BATCH_SIZE=int(sys.argv[n+1]);
    elif sys.argv[n]=='--ncod':
        NCOD=int(sys.argv[n+1]);
    elif sys.argv[n]=='--patience':
        patience=int(sys.argv[n+1]);
    elif sys.argv[n]=='--seed':
        seed_number=int(sys.argv[n+1]);
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];
        
print('   dataset_base_dir:',dataset_base_dir)
print('dataset_labels_file:',dataset_labels_file)
print('       dataset_name:',dataset_name)
print('                  K:',K)
print('  enable_stratified:',enable_stratified)
print('             EPOCAS:',EPOCAS)
print('         BATCH_SIZE:',BATCH_SIZE)
print('               NCOD:',NCOD)
print('           patience:',patience)
print('        seed_number:',seed_number)
print('    output_base_dir:',output_base_dir)

   dataset_base_dir: /media/fernando/Expansion/DATASET/TESE/BER/BER2024/BER2024-FUSION
dataset_labels_file: train.csv
       dataset_name: ber2024-fusion
                  K: 5
  enable_stratified: True
             EPOCAS: 50
         BATCH_SIZE: 1024
               NCOD: 15
           patience: 10
        seed_number: 0
    output_base_dir: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4_fusion_1


# Set seed of random variables


In [9]:
np.random.seed(seed_number)
tf.keras.utils.set_random_seed(seed_number);

# Setting the cross-validation kfold


In [10]:
from sklearn.model_selection import KFold, StratifiedKFold

output_dir = os.path.join(output_base_dir,dataset_name,'cross-validation','cls_ncod'+str(NCOD));

if enable_stratified:
    kf = StratifiedKFold(n_splits = K, shuffle = True, random_state = seed_number);
else:
    kf  = KFold(n_splits = K, shuffle=True, random_state=seed_number); 

# Loading data of dataset

In [11]:
# Load filenames and labels
train_val_df = pd.read_csv(os.path.join(dataset_base_dir,dataset_labels_file));
print(train_val_df)
# Setting labels
Y   = train_val_df[['label']];
L=np.shape(Y)[0];

             d0        d1        d2        d3            d4        d5  \
0      0.224582  0.133591  0.225396  0.416430  1.150091e-01  0.208645   
1      0.037114  0.187126  0.258345  0.517415  1.151926e-01  0.126007   
2      0.007186  0.297487  0.224354  0.470973  2.066710e-01  0.182703   
3      0.367856  0.171388  0.282665  0.178091  5.219421e-01  0.139584   
4      0.276513  0.140360  0.482942  0.100184  1.138967e-01  0.141131   
...         ...       ...       ...       ...           ...       ...   
56954  0.999740  0.000158  0.000079  0.000022  9.998608e-01  0.000139   
56955  0.000441  0.999499  0.000023  0.000037  9.647324e-05  0.999873   
56956  0.000008  0.000045  0.000071  0.999876  4.592921e-07  0.000004   
56957  0.936142  0.062899  0.000425  0.000533  9.808379e-01  0.019134   
56958  0.003416  0.996173  0.000210  0.000201  2.694909e-05  0.999970   

                 d6            d7        d8        d9       d10       d11  \
0      2.310063e-01  4.453400e-01  0.322304  0

# Auxiliar function

In [12]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

# Creating output directory

In [13]:
os.makedirs(output_dir,exist_ok = True) 
print(output_dir)


/media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4_fusion_1/ber2024-fusion/cross-validation/cls_ncod15


# Creating output status file

In [14]:
json_status_path=os.path.join(output_dir,fold_status_file);

# Cross-validation

In [15]:
import FusionEmotion4Lib.lib_model as mpp
#import matplotlib.pyplot as plt

list_train_index=[];
list_val_index=[];
for train_index, val_index in kf.split(np.zeros(L),Y):
    list_train_index.append(train_index);
    #print('train_index:',train_index);
    list_val_index.append(val_index);
    #print('val_index:',val_index);

data_fold =  {'val_categorical_accuracy': [],
              'val_loss': [], 
              'train_categorical_accuracy': [],
              'train_loss': [] 
             };

fold_var=1;

if os.path.isfile(json_status_path):
    # Read JSON file
    with open(json_status_path) as data_file:
        data_fold = json.load(data_file)
        fold_var=len(data_fold['val_loss'])+1;

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()

while fold_var<=K:
    training_df   = train_val_df.iloc[list_train_index[fold_var-1]];
    validation_df = train_val_df.iloc[list_val_index[fold_var-1]];

    print('\nFold:',fold_var);
    
    # CREATE NEW MODEL
    model = mpp.create_model(   load_weights=False,
                                file_of_weight='',
                                ncod=NCOD);
    
    model.summary();
    
    training_data_array   = training_df.iloc[:,0:12].values;
    training_y_label      = training_df.iloc[:,12].values;
    
    validation_data_array = validation_df.iloc[:,0:12].values;
    validation_y_label    = validation_df.iloc[:,12].values;
    
    training_y   = label_encoder.fit_transform(training_y_label);
    validation_y = label_encoder.fit_transform(validation_y_label);
    
    training_y_categorical   = to_categorical(training_y);
    validation_y_categorical = to_categorical(validation_y);

    print('train:',training_data_array.shape,training_y_categorical.shape)
    print('  val:',validation_data_array.shape,validation_y_categorical.shape)
    
    #STEPS_BY_EPOCHS=len(training_dataset);
    
    # COMPILE NEW MODEL
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['categorical_accuracy'])
    
    # CREATE CALLBACKS
    best_model_file=os.path.join(output_dir,get_model_name(fold_var));
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=best_model_file, 
                                                    save_weights_only=True,
                                                    monitor='val_loss', 
                                                    save_best_only=True, 
                                                    verbose=1);
    # Definindo o callback EarlyStopping
    early_stopping = tf.keras.callbacks.EarlyStopping(  monitor='val_loss', 
                                                        patience=patience,  
                                                        verbose=1, 
                                                        restore_best_weights=False);
    
    log_dir = os.path.join(output_dir,"logs","fit", "fold"+str(fold_var)+'_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S"));
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    
    # There can be other callbacks, but just showing one because it involves the model name
    # This saves the best model
    # FIT THE MODEL
    history = model.fit(training_data_array,training_y_categorical,
                        #steps_per_epoch=STEPS_BY_EPOCHS,
                        epochs=EPOCAS,
                        batch_size=BATCH_SIZE,
                        validation_data=(validation_data_array,validation_y_categorical),
                        callbacks=[checkpoint,early_stopping,tensorboard_callback],
                        verbose=1
                       );
    
    #PLOT HISTORY
    mpp.save_model_history(history,
                           os.path.join(output_dir,"historical_"+str(fold_var)+".csv"),
                           show=False,
                           labels=['categorical_accuracy','loss']);
    
    # LOAD BEST MODEL to evaluate the performance of the model
    model.load_weights(best_model_file);

    print("\n")
    
    # Evaluate training
    results = model.evaluate(training_data_array,training_y_categorical);
    results = dict(zip(model.metrics_names,results));
    print("Training:\n",results,"\n\n");
    data_fold['train_categorical_accuracy'].append(results['categorical_accuracy']);
    data_fold['train_loss'].append(results['loss']);

    # Evaluate validation
    results = model.evaluate(validation_data_array,validation_y_categorical)
    results = dict(zip(model.metrics_names,results))
    print("Validation:\n",results,"\n\n");
    data_fold['val_categorical_accuracy'].append(results['categorical_accuracy'])
    data_fold['val_loss'].append(results['loss'])

    data_fold['number_of_parameters']=mpp.get_model_parameters(model);
    data_fold['ncod']=NCOD;
    data_fold['epochs']=EPOCAS;
    data_fold['patience']=patience;
    data_fold['seed']=seed_number;
    
    # Data fold
    with open(json_status_path, 'w') as f:
        json.dump(data_fold, f,indent=4);

    tf.keras.backend.clear_session()
    
    fold_var += 1


Fold: 1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                195       
                                                                 
 dense_1 (Dense)             (None, 4)                 64        
                                                                 
Total params: 259 (1.01 KB)
Trainable params: 259 (1.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2024-08-08 17:34:32.409509: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-08 17:34:32.409927: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-08 17:34:32.410251: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

train: (45567, 12) (45567, 4)
  val: (11392, 12) (11392, 4)
Epoch 1/50


I0000 00:00:1723149285.117078  114310 service.cc:145] XLA service 0x7d2cb077b230 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723149285.117406  114310 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce MX150, Compute Capability 6.1
2024-08-08 17:34:45.127592: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-08 17:34:45.150700: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
I0000 00:00:1723149285.238264  114310 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


31/45 [===================>..........] - ETA: 0s - loss: 1.3729 - categorical_accuracy: 0.2574
Epoch 1: val_loss improved from inf to 1.23415, saving model to /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4_fusion_1/ber2024-fusion/cross-validation/cls_ncod15/model_1.h5
45/45 [==============================] - 13s 17ms/step - loss: 1.3433 - categorical_accuracy: 0.3078 - val_loss: 1.2342 - val_categorical_accuracy: 0.4874
Epoch 2/50
43/45 [===========================>..] - ETA: 0s - loss: 1.1305 - categorical_accuracy: 0.6595
Epoch 2: val_loss improved from 1.23415 to 1.00828, saving model to /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4_fusion_1/ber2024-fusion/cross-validation/cls_ncod15/model_1.h5
45/45 [==============================] - 0s 6ms/step - loss: 1.1265 - categorical_accuracy: 0.6651 - val_loss: 1.0083 - val_categorical_accuracy: 0.8433
Epoch 3/50
30/45 [===================>..........] - ETA: 0s - loss: 0.9401 - categorical_accuracy: 0.8885
Epoch 3: 

In [ ]:
import json


data_fold['mean_val_categorical_accuracy'] = np.mean(data_fold['val_categorical_accuracy']);
data_fold['std_val_categorical_accuracy']  = np.std(data_fold['val_categorical_accuracy']);

data_fold['mean_val_loss'] = np.mean(data_fold['val_loss']);
data_fold['std_val_loss']  = np.std(data_fold['val_loss']);

data_fold['mean_train_categorical_accuracy'] = np.mean(data_fold['train_categorical_accuracy']);
data_fold['std_train_categorical_accuracy']  = np.std(data_fold['train_categorical_accuracy']);

data_fold['mean_train_loss'] = np.mean(data_fold['train_loss']);
data_fold['std_train_loss']  = np.std(data_fold['train_loss']);

print(json.dumps(data_fold, indent=4))

# Data fold
with open(json_status_path, 'w') as f:
    json.dump(data_fold, f,indent=4);